In [27]:
import os
from langchain_community.document_loaders import TextLoader, WebBaseLoader, PDFMinerLoader
from langchain.text_splitter import MarkdownHeaderTextSplitter, RecursiveCharacterTextSplitter
from langchain.docstore.document import Document
from langchain_community.vectorstores import SKLearnVectorStore
from langchain.vectorstores import Chroma
from groq import Groq
from langchain.chains.combine_documents.map_reduce import MapReduceDocumentsChain
from langchain import LLMChain
from langchain_groq import ChatGroq
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_huggingface import HuggingFaceEmbeddings
import requests
import re


In [ ]:
os.environ["GROQ_API_KEY"] = "groq_API_key"

In [29]:

url = "https://www.parliament.go.ke/sites/default/files/2017-05/The_Constitution_of_Kenya_2010.pdf"
r = requests.get(url)
with open("constitution.pdf", "wb") as f:
    f.write(r.content)
    
loader = PDFMinerLoader("constitution.pdf")
docs_list = loader.load() 

In [30]:
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=1500,
    chunk_overlap=500,
    separators=["ARTICLE", "CHAPTER", "\n\n"])

doc_splits = text_splitter.split_documents(docs_list)

In [31]:
#Embedding the documents and storing  them in a vector store
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
vectorstore = Chroma.from_documents(
    documents=doc_splits,          
    embedding=embeddings,
    persist_directory="constitution_index"
)

retriever = vectorstore.as_retriever(
    search_kwargs={
        "k": 5
    }
)

In [ ]:
llm = ChatGroq(
    model="llama-3.1-8b-instant",
    temperature = 0
    max_tokens=500
)

In [33]:
prompt = PromptTemplate(
    template="""
You are an assistant for question-answering Kenyan constitution tasks.
Use the following documents to answer the question.
If you don't know the answer, just say that you don't know.
Question: {question}
Documents: {documents}
Answer:
""",
input_variables=["question", "documents"]
)


In [34]:
rag_chain = prompt | llm | StrOutputParser()

In [35]:
class RAGApplication:
    def __init__(self, retriever, rag_chain):
        self.retriever = retriever
        self.rag_chain = rag_chain

    def run(self, question):
        documents = self.retriever.invoke(question)
        doc_texts = "\n".join([doc.page_content for doc in documents])
        
        # Get the answer from the LLM
        answer = self.rag_chain.invoke({"question": question, "documents": doc_texts})
        return answer


In [44]:
rag_app = RAGApplication(retriever, rag_chain)

question = "What are grounds for losing a seat?"
answer = rag_app.run(question)

print("Answer:", answer)

Answer: According to Article 194 of the Kenyan Constitution, the grounds for losing a seat in the county assembly are as follows:

- If the member dies (194(1)(a))
- If the member is absent from eight sittings of the assembly without permission and is unable to offer a satisfactory explanation (194(1)(b))
- If the member is removed from office under the Constitution or legislation enacted under Article 80 (194(1)(c))
- If the member resigns in writing addressed to the speaker of the assembly (194(1)(d))
- If the member is elected as a member of a political party and resigns from the party (194(1)(e)(i))
- If the member is elected as an independent candidate and joins a political party (194(1)(e)(ii))
- At the end of the term of the assembly (194(1)(f))
- If the member becomes disqualified for election on grounds specified in Article 193(2) (194(1)(g))


In [37]:
def ask(question):
    answer = rag_app.run(question)
    print("Answer:", answer)



In [38]:
ask("What are the children rights ?")

Answer: According to the Constitution of Kenya, 2010, the children's rights are as follows:

1. To a name and nationality from birth.
2. To free and compulsory basic education.
3. To basic nutrition, shelter, and health care.
4. To be protected from:
   - Abuse
   - Neglect
   - Harmful cultural practices
   - All forms of violence
   - Inhuman treatment and punishment
   - Hazardous or exploitative labor


In [45]:
ask("What is the term limit of a president?")

Answer: According to the Kenyan constitution, a person shall not hold office as President for more than two terms.


In [42]:
ask("What are the types of lands recognized?")

Answer: Based on Article 63 of the Constitution of Kenya, 2010, the types of lands recognized are:

1. (c) land transferred to the State by way of sale, reversion or surrender;
2. (d) land in respect of which no individual or community ownership can be established by any legal process;
3. (e) land in respect of which no heir can be identified by any legal process;
4. (f) all minerals and mineral oils as defined by law;
5. (g) government forests other than forests to which Article 63 (2) (d) (i) applies, government game reserves, water catchment areas, national parks, government animal sanctuaries, and specially protected areas;
6. (h) all roads and thoroughfares provided for by an Act of Parliament;
7. (i) all rivers, lakes and other water bodies as defined by an Act of Parliament;
8. (j) the territorial sea, the exclusive economic zone and the sea bed;
9. (k) the continental shelf;
10. (l) all land between the high and low water marks;
11. (m) any land not classified as private or com

In [43]:
ask("What institutions are responsible for land management")

Answer: Based on the provided documents, the institutions responsible for land management are:

1. National Land Commission (Article 67 of the Constitution and the functions listed below)

The National Land Commission has the following functions:

- (a) to manage public land on behalf of the national and county governments
- (b) to recommend a national land policy to the national government
- (c) to advise the national government on a comprehensive programme for the registration of title in land throughout Kenya
- (d) to conduct research related to land and the use of natural resources, and make recommendations to appropriate authorities
- (e) to initiate investigations, on its own initiative or on a complaint, into present or historical land injustices, and recommend appropriate redress
- (f) to encourage the application of traditional dispute resolution mechanisms in land conflicts
- (g) to assess tax on land and premiums on immovable property in any area designated by law
- (h) to m